In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, trim, concat_ws, to_date
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, DateType
import time
spark = (
    SparkSession.builder.master("local[*]").appName("pyspark_dataframe").getOrCreate()
)
# spark = SparkSession.builder.getOrCreate()

from pyspark import SparkContext
from pyspark.streaming import StreamingContext
sc = spark.sparkContext

25/03/10 19:30:25 WARN Utils: Your hostname, iadb-10 resolves to a loopback address: 127.0.1.1; using 172.20.104.46 instead (on interface wlp44s0f0)
25/03/10 19:30:25 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/10 19:30:26 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [70]:
time.sleep(2)

### Ejercicio 1 ###

Con el archivo Turismo.csv lleva a cabo las siguientes acciones utilizando Dataframes de SPARK.

En este ejercicio debes crear el archivo: Ejercicio1.ipynb  con los pasos llevados a cabo y subirlo a moodle.
En el archivo que entregues que se vean todos los resultados de las operaciones.


    1. Cargar los datos en un DataFrame de Spark infiriendo el esquema. HECHO
    2. Mostrar el esquema del DataFrame y los primeros 5 registros. HECHO
    3. Convertir la columna "FECHA" en formato de fecha (dd/MM/yyyy) y mostrar el esquema. HECHO
    4. Listar las 3 visitas con la mayor puntuación de: SATISFACCION_GLOBAL. HECHO
    5. Contar cuántas visitas se realizaron en cada mes del año y mostrarlas en orden descendente. HECHO
    6. Filtrar las visitas realizadas en septiembre y mostrar sólo los campos: VISITA y SATISFACCION_GLOBAL. HECHO
    7. Reemplazar valores nulos en la columna: LO_PEOR con el valor: "Sin comentarios". HECHO
    8. Crear una nueva columna llamada SATISFACCION_PROMEDIO que sea el promedio de las 3 categorías de satisfacción: SATISFACCION_INFOR, SATISFACCION_RECORRIDO, SATISFACCION_GUIA. Debes crear tu propia función de usuario y registrarla como una función de Spark. Con SQL y sin SQL.
    9. Guardar el resultado final en formato .csv de tal forma que si el archivo ya existe que lo sobreescriba.
    

In [6]:
!head ./datos/Turismo.csv

ID;VISITA;MES;FECHA;SATISFACCION_INFOR;SATISFACCION_RECORRIDO;SATISFACCION_GUIA;SATISFACCION_GLOBAL;LO_MEJOR;LO_PEOR;OPINION;SUGERENCIAS
12;LEYENDAS, CASAS ENCANTADAS Y LUCES EN  NAVIDAD CON ILSE;DICIEMBRE;17/12/2023;10;10;10;10;7 chimeneas ;Nada;Mejor de lo que esperaba;Nada, todo perfecto 
19;MADRID EN FEMENINO;AGOSTO;12/08/2023;8;8;9;9;Al restaurante Sobrinos de Botín;El mapa final, hacia muchísimo sol;Mejor de lo que esperaba;Combinar mas las explicaciones de las mujeres con las explicaciones sobre la historia y tmb explicar las la conexión de las mujeres con los lugares que se visitan
3;MADRID EN FEMENINO;SEPTIEMBRE;02/09/2023;9;10;10;10;Amabilidad y conocimiento de la guía turística;;Mejor de lo que esperaba;Más visitas y en distintos horarios
19;LA LATINA Y EL MUSEO DE LOS ORIGENES;SEPTIEMBRE;16/09/2023;10;10;10;10;Beatriz Galindo latina ;Nada;Mejor de lo que esperaba;Me han gustado mucho 
35;LA LATINA Y EL MUSEO DE LOS ORIGENES;OCTUBRE;22/10/2023;7;8;8;8;Bien ;Bien ;Mejor de lo

Apartado 1

In [10]:
dfCSV = spark.read.options(sep=";", header=True, inferSchema=True).csv("datos/Turismo.csv")


Apartado 2

In [11]:
dfCSV.printSchema()

root
 |-- ID: string (nullable = true)
 |-- VISITA: string (nullable = true)
 |-- MES: string (nullable = true)
 |-- FECHA: string (nullable = true)
 |-- SATISFACCION_INFOR: integer (nullable = true)
 |-- SATISFACCION_RECORRIDO: integer (nullable = true)
 |-- SATISFACCION_GUIA: integer (nullable = true)
 |-- SATISFACCION_GLOBAL: integer (nullable = true)
 |-- LO_MEJOR: string (nullable = true)
 |-- LO_PEOR: string (nullable = true)
 |-- OPINION: string (nullable = true)
 |-- SUGERENCIAS: string (nullable = true)



In [12]:
dfCSV.show(5)

+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| ID|              VISITA|       MES|     FECHA|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|            LO_MEJOR|             LO_PEOR|             OPINION|         SUGERENCIAS|
+---+--------------------+----------+----------+------------------+----------------------+-----------------+-------------------+--------------------+--------------------+--------------------+--------------------+
| 12|LEYENDAS, CASAS E...| DICIEMBRE|17/12/2023|                10|                    10|               10|                 10|        7 chimeneas |                Nada|Mejor de lo que e...|Nada, todo perfecto |
| 19|  MADRID EN FEMENINO|    AGOSTO|12/08/2023|                 8|                     8|                9|                  9|Al restaurante So...

Apartado 3

In [22]:
dfCSV = dfCSV.withColumn("Fecha", to_date(dfCSV.FECHA, "dd/MM/yyyy"))

In [23]:
dfCSV.printSchema()

root
 |-- ID: string (nullable = true)
 |-- VISITA: string (nullable = true)
 |-- MES: string (nullable = true)
 |-- Fecha: date (nullable = true)
 |-- SATISFACCION_INFOR: integer (nullable = true)
 |-- SATISFACCION_RECORRIDO: integer (nullable = true)
 |-- SATISFACCION_GUIA: integer (nullable = true)
 |-- SATISFACCION_GLOBAL: integer (nullable = true)
 |-- LO_MEJOR: string (nullable = true)
 |-- LO_PEOR: string (nullable = true)
 |-- OPINION: string (nullable = true)
 |-- SUGERENCIAS: string (nullable = true)



Apartado 4

In [28]:
dfCSV.sort(dfCSV.SATISFACCION_GLOBAL.desc()).show(3)

+---+--------------------+---------+-----+------------------+----------------------+-----------------+-------------------+--------------------+-------+--------------------+--------------------+
| ID|              VISITA|      MES|Fecha|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|            LO_MEJOR|LO_PEOR|             OPINION|         SUGERENCIAS|
+---+--------------------+---------+-----+------------------+----------------------+-----------------+-------------------+--------------------+-------+--------------------+--------------------+
| 15|LA HISTORIA IMPRE...|   AGOSTO| NULL|                10|                    10|               10|                 10|                NULL|   NULL|       Como esperaba|Amplia nuevas vis...|
|  3|    MADRID HISTÓRICO|    MARZO| NULL|                10|                    10|               10|                 10|El conjunto está ...|   NULL|Mejor de lo que e...|                NULL|
| 20|TRADICIONES NAVID...|DICI

Apartado 5

In [37]:
dfCSV.groupBy("MES").count().orderBy(col('count').desc()).show()

+--------------------+-----+
|                 MES|count|
+--------------------+-----+
|             OCTUBRE|   41|
|          SEPTIEMBRE|   36|
|               ABRIL|   34|
|           DICIEMBRE|   32|
|              AGOSTO|   31|
|           NOVIEMBRE|   29|
|               JULIO|   26|
|               JUNIO|   21|
|             FEBRERO|   18|
|                Mayo|   14|
|              ENERO |   13|
|                NULL|    6|
|               MARZO|    3|
|Creo que podrían ...|    1|
+--------------------+-----+



Hay un pequeño error en alguna columna y hay unos nulos.

Apartado 6

In [38]:
dfCSV.select("VISITA","SATISFACCION_GLOBAL").filter(dfCSV.MES == "SEPTIEMBRE").show()

+--------------------+-------------------+
|              VISITA|SATISFACCION_GLOBAL|
+--------------------+-------------------+
|  MADRID EN FEMENINO|                 10|
|LA LATINA Y EL MU...|                 10|
|  MADRID EN FEMENINO|                  9|
|   MADRID MONUMENTAL|                 10|
|   MADRID MONUMENTAL|                 10|
|  MADRID EN FEMENINO|                  6|
|  MADRID EN FEMENINO|                  6|
|LA HISTORIA IMPRE...|                 10|
|LA LATINA Y EL MU...|                  9|
|  MADRID EN FEMENINO|                  8|
|LA HISTORIA IMPRE...|                 10|
|   MADRID MONUMENTAL|                 10|
|  MADRID EN FEMENINO|                  8|
|HISTORIAS Y LEYEN...|                  9|
|HISTORIAS Y LEYEN...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA NUEVA PLAZA DE...|                 10|
|LA LATINA Y EL MU...|                 10|
+----------

Apartado 7

In [49]:
dfCSV=dfCSV.na.fill("Sin comentarios", subset=["LO_PEOR"])

In [50]:
dfCSV.filter(dfCSV.LO_PEOR.isNull()).show()

+---+------+---+-----+------------------+----------------------+-----------------+-------------------+--------+-------+-------+-----------+
| ID|VISITA|MES|Fecha|SATISFACCION_INFOR|SATISFACCION_RECORRIDO|SATISFACCION_GUIA|SATISFACCION_GLOBAL|LO_MEJOR|LO_PEOR|OPINION|SUGERENCIAS|
+---+------+---+-----+------------------+----------------------+-----------------+-------------------+--------+-------+-------+-----------+
+---+------+---+-----+------------------+----------------------+-----------------+-------------------+--------+-------+-------+-----------+



Apartado 8

Apartado 9

In [ ]:
# Por motivos desconocidos esto crea una subcarpeta llamada Turismo.csv en la carpeta datos 
#dfCSV.write.mode("overwrite").csv("datos/Turismo.csv")

# Eto funciona
dfCSV.toPandas().to_csv('datos/Turismo.csv')


### Ejercicio 2 ###

Desarrolla una aplicación en PySpark Streaming que cuente la frecuencia de palabras en tiempo real.
La entrada de datos se proporcionará manualmente a través de un servidor de socket. Y la salida de datos se mostrará por consola.

    1. Mostrar la cuenta de las palabras utilizando ventanas fijas de 2 minutos de duración. 

Muestra los resultados sin que se trunquen para que se vea la información completa.

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split


import time
spark = (
    SparkSession.builder.master("local[*]").appName("pyspark_dataframe").getOrCreate()
)
# spark = SparkSession.builder.getOrCreate()

from pyspark import SparkContext
from pyspark.streaming import StreamingContext


sc = spark.sparkContext


ssc = StreamingContext(sc, 1)

lines = spark \
    .readStream \
    .format("socket") \
    .option("host", "localhost") \
    .option("port", 9999) \
    .load()

# Split the lines into words
words = lines.select(
   explode(
       split(lines.value, " ")
   ).alias("word")
)

# Generate running word count
wordCounts = words.groupBy("word").count()

query = wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start()


ssc.awaitTermination()

25/03/10 19:53:51 WARN Utils: Your hostname, iadb-10 resolves to a loopback address: 127.0.1.1; using 172.20.104.46 instead (on interface wlp44s0f0)
25/03/10 19:53:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/10 19:53:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
/home/iabd/anaconda3/envs/IABD3_12/lib/python3.12/site-packages/pyspark/streaming/context.py:72: FutureWarning: DStream is deprecated as of Spark 3.4.0. Migrate to Structured Streaming.
  warnings.warn(
25/03/10 19:53:52 WARN TextSocketSourceProvider: The socket source should not be used for production applications! It does not support recovery.
25/03/10 19:53:53 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/

-------------------------------------------
Batch: 0
-------------------------------------------
+----+-----+
|word|count|
+----+-----+
+----+-----+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------+-----+
|    word|count|
+--------+-----+
|       o|    1|
|   menos|    1|
|     mas|    1|
|funciona|    1|
+--------+-----+



ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/iabd/anaconda3/envs/IABD3_12/lib/python3.12/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
RuntimeError: reentrant call inside <_io.BufferedReader name=64>

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/iabd/anaconda3/envs/IABD3_12/lib/python3.12/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/iabd/anaconda3/envs/IABD3_12/lib/python3.12/site-packages/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving
ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/home/i

Py4JError: An error occurred while calling o29.awaitTermination

25/03/10 19:54:03 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors
